In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX

In [2]:
directory = joinpath(homedir(), "research", "GADS", "Events")
files = glob("*.csv", directory)
x = Dict()
i = 1
for f in files
    df = DataFrame(CSV.File(f, select=["EventStartDT", "EventEndDT", "UnitID", "UnitTypeCode", "EventTypeCode", "OutageContinuationFlag"]))
    x[i] = df
    i +=1
end
merged = reduce(vcat, values(x))
sort!(merged, :EventStartDT)

Row,EventEndDT,EventStartDT,EventTypeCode,OutageContinuationFlag,UnitID,UnitTypeCode
,String31,DateTime,String3,Int64,Int64,Int64
1,2013-01-05T06:54:00.0,2013-01-01T04:00:00,RS,2,5408,100
2,2013-01-02T22:16:00.0,2013-01-01T04:00:00,RS,2,5423,500
3,2013-01-02T22:21:00.0,2013-01-01T04:00:00,RS,2,5424,500
4,2013-01-12T22:05:00.0,2013-01-01T04:00:00,RS,2,5427,500
5,2013-01-26T19:10:00.0,2013-01-01T04:00:00,RS,2,5415,300
6,2013-01-28T17:16:00.0,2013-01-01T04:00:00,RS,2,11240,300
7,2013-01-02T20:57:00.0,2013-01-01T04:00:00,RS,2,5426,500
8,2013-02-01T04:00:00.0,2013-01-01T04:00:00,RS,2,5420,300
9,2013-01-01T18:52:00.0,2013-01-01T04:00:00,RS,2,5407,100


In [3]:
fname = joinpath(homedir(), "research", "GADS", "GADS_Unit_2023_20240422.xlsx")
units = DataFrame(XLSX.readtable(fname, "Sheet1"));

In [15]:
event_gen_ids = unique(merged[!, "UnitID"])
all_gen_ids = unique(units[!, "UnitID"])

not_in_event_gen_ids = setdiff(all_gen_ids, event_gen_ids)

# units[units.UnitID in not_in_event_gen_ids, !]

not_in_events_df = filter(row -> row[:UnitID] in not_in_event_gen_ids, units)

Row,UnitID,AliasID,CommercialDT_grp,ID_SK,NERCID_AliasID,RatingMW_grp,RegionCode,StateName,UnitRetireDT_grp,UnitTimeZoneCode,UnitTypeCode,UnitTypeCodeDesc,RatingMW_BrochureGroup
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,3009,0xAFFAEAA4F1A21A6726FCD1AB86AD5856,1,16935,0x53561B76F369CF1A9078EBB86BA57B02,2,SERC,Mississippi,Retired,CPT,100,Fossil-Steam,100-199
2,1,0xC92C0BABDC764D8674BCEA14A55D867D,3,12664,0xD2FB8E0971BCA5276F5AC35BE03FEC06,2,WECC,California,Retired,PPT,860,Co-generator Block,All
3,167,0x66B52E915FD1FA80AF777CF2570A71CC,1,18748,0x396460EBE8CCC4671276C7FE8C3F4DF8,1,RFC,West Virginia,Retired,EPT,100,Fossil-Steam,<100
4,2496,0x0EEBFC360E251DD76E174F2E77C9F94B,1,23058,0xB2E0F11027E5BED71F0A7522909A82E5,1,SERC,North Carolina,Retired,EPT,300,Gas Turbine/Jet Engine (Simple Cycle Operation),<50
5,10583,0x08F0B95FD319FA1A0AAF7A4026C6FC57,5,22876,0x0E5174DF8B8FDFF3A3B4A6DA534C4D23,3,NPCC,New York,missing,EPT,850,Combined Cycle Block,All
6,2493,0x62913A81BB0D8572105C4F6AE0FB4FAB,1,23055,0xB2E0F11027E5BED71F0A7522909A82E5,1,SERC,North Carolina,Retired,EPT,300,Gas Turbine/Jet Engine (Simple Cycle Operation),<50
7,3010,0x2B69F2E81CC22760EAD0489D372CDA72,1,16936,0x53561B76F369CF1A9078EBB86BA57B02,2,SERC,Mississippi,Retired,CPT,100,Fossil-Steam,100-199
8,7971,0x391D84E6378D8A29B1911A144F7E00CC,1,17106,0x5818659E035F675B37279D80A51F6DC1,1,MRO,Texas,Retired,CPT,300,Gas Turbine/Jet Engine (Simple Cycle Operation),<50
9,186,0x968B2A08F4E829F662DD18762E80B8EC,1,18767,0x396460EBE8CCC4671276C7FE8C3F4DF8,1,RFC,Maryland,Retired,EPT,100,Fossil-Steam,<100


In [30]:
a = not_in_events_df[!, :UnitRetireDT_grp]
count(x -> true, skipmissing(a))

117

In [28]:
gen_ids = unique(merged[!, "UnitID"])
merged.CommercialDT_grp = missings(Int, nrow(merged))
merged.RatingMW_grp = missings(Int, nrow(merged))
merged.StateName = missings(String, nrow(merged))

myDict = Dict()

for u in gen_ids
    val = units[units.UnitID .== u, :CommercialDT_grp][1]
    vec = merged[merged.UnitID .== u, :CommercialDT_grp]
    new_vec = fill(val, length(vec))
    merged[merged.UnitID .== u, :CommercialDT_grp] = new_vec

    val = units[units.UnitID .== u, :RatingMW_grp][1]
    vec = merged[merged.UnitID .== u, :RatingMW_grp]
    new_vec = fill(val, length(vec))
    merged[merged.UnitID .== u, :RatingMW_grp] = new_vec

    val = units[units.UnitID .== u, :StateName][1]
    vec = merged[merged.UnitID .== u, :StateName]
    new_vec = fill(val, length(vec))
    merged[merged.UnitID .== u, :StateName] = new_vec

    myDict[u] = merged[merged.UnitID .== u, :]
end

In [32]:
for u in gen_ids
    df = myDict[u]
    unit_type_code = df[1, :UnitTypeCode]
    commercial_dt_grp = df[1, :CommercialDT_grp]
    state_name = df[1, :StateName]
    rating = df[1, :RatingMW_grp]

    fname = @sprintf "gen_%d_type_%d_dtgrp_%d_rating_%d_state_%s_raw.csv" u unit_type_code commercial_dt_grp rating state_name
    path = joinpath(homedir(), "research", "GADS_Processed", "raw_data", fname)
    CSV.write(path, df)
end